**Scenario**
1. CJ and Pavol are collaborating on a paleontology project: imaging 3 teeth, sabre-tooth, shark and dinosaur
1. CJ is working at a (fictional) NSLS-II beamline overnight
1. CJ is taking full-field image data for tomographic analysis using a strip detector (so he gets one slice of the 3D projection)


1. The instrument scientist initializes everything

In [1]:
import os
import time

import bluesky.plans as bp
import dxchange
import numpy as np
import tomopy
from bluesky.run_engine import RunEngine
from xpdan.vend.callbacks import LiveTable
from ophyd.sim import SynSignal, hw, SynSignalWithRegistry
from xpdan.vend.callbacks.zmq import Publisher
from xpdconf.conf import glbl_dict

# create fake hardware (motors and detector)
hw = hw()
fname = os.path.expanduser("tooth.h5")

proj, flat, dark, theta = dxchange.read_aps_32id(fname, sino=(0, 1))

proj = tomopy.normalize(proj, flat, dark)

rot_center = tomopy.find_center(proj, theta, init=290, ind=0, tol=0.5)
proj2 = proj
theta_motor = hw.motor1
theta_motor.kind = "hinted"


class FullField:
    def __call__(self, *args, **kwargs):
        v = theta_motor.get()[0]
        out = proj2[int(v), :, :]
        time.sleep(.5)
        return out

f = FullField()
det = SynSignalWithRegistry(f, name="img", labels={"detectors"},
                            save_path='/home/christopher/dev/provenance-driven-ldrd/demo/raw_data')
det.kind = "hinted"

# create run engine, link with ZMQ system
RE = RunEngine()
p = Publisher(glbl_dict["inbound_proxy_address"], prefix=b"raw")
t = RE.subscribe(p)

/home/christopher/mc/envs/dp_dev/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


1. CJ builds a scan that is a list of sample angles where he wants to collect data.  Because he is clever he doesn't do these in increasing angle order but bisects the angle each time to get the most angular coverage in the shortest time.

In [2]:
# build tomo scan locations
def build_scan():
    l = [0., 90.]
    for i in range(8):
        ll = l.copy()
        interval = sorted(set(ll))[1] / 2
        for lll in ll:
            j = lll + interval
            j = round(j, 0)
            if j not in l and j <= 180:
                l.append(j)
    return l


1. Now CJ is ready to set up his first scan so he builds a scan-plan. He will use the bluesky "list_scan" which takes the list of detectors, the motor that will be driven between each shot (in this case the ``theta_motor``) and the list of values to give to the motor (in this case angle in degrees).  Finally, the scan takes a dictionary of metadata (``md``) that will be stored in the run-start header.

In [3]:
shots = 16
base_metadata = {
            "tomo": {
                "type": "full_field",
                "rotation": "motor1",
                "center": rot_center,
            },
            'bt_piLast': 'Wright',
            'analysis_stage': 'raw',
            'sample_type': 'tooth',
        }
saber_tooth_md = {**base_metadata, **{'sample_name': 'saber tooth'}}
shark_md = {**base_metadata, **{'sample_name': 'shark tooth'}}
dinosaur_md = {**base_metadata, **{'sample_name': 'dinosaur tooth'}}

saber_tooth_scan = bp.list_scan(
        [det],
        theta_motor,
        build_scan()[:shots],
        md=saber_tooth_md,
    )
dinosaur_scan = bp.list_scan(
        [det],
        theta_motor,
        build_scan()[:shots],
        md=dinosaur_md,
    )
# etc.

1. CJ runs the saber tooth and shark scans, then, at 3am, he loads the dinosaur tooth and starts the scan

In [4]:
RE(dinosaur_scan, LiveTable(['motor1']))

+-----------+------------+------------+
|   seq_num |       time |     motor1 |
+-----------+------------+------------+
|         1 | 11:35:03.4 |      0.000 |
|         2 | 11:35:03.9 |     90.000 |
|         3 | 11:35:04.4 |     45.000 |
|         4 | 11:35:04.9 |    135.000 |
|         5 | 11:35:05.4 |     22.000 |
|         6 | 11:35:05.9 |    112.000 |
|         7 | 11:35:06.4 |     68.000 |
|         8 | 11:35:06.9 |    158.000 |
|         9 | 11:35:07.5 |     11.000 |
|        10 | 11:35:08.0 |    101.000 |
|        11 | 11:35:08.5 |     56.000 |
|        12 | 11:35:09.0 |    146.000 |
|        13 | 11:35:09.5 |     33.000 |
|        14 | 11:35:10.1 |    123.000 |
|        15 | 11:35:10.6 |     79.000 |
|        16 | 11:35:11.1 |    169.000 |
+-----------+------------+------------+
generator list_scan ['4c9cba94'] (scan num: 1)


('4c9cba94-c2a3-465e-82c3-b5938f633315',)